<a href="https://colab.research.google.com/github/jainstuti/Tweets_classification/blob/main/NB_TMLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#importing dataset
dataset = pd.read_csv('Twitter_Data.csv')

In [3]:
dataset=dataset.head(90000)

In [4]:
dataset.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [5]:
# to get the values in categorical data and their counts in the data
dataset['category'].value_counts()

 1.0    40059
 0.0    30375
-1.0    19566
Name: category, dtype: int64

In [6]:
dataset.describe()

,category
count,90000.000000
mean,0.227700
std,0.781447
min,-1.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [7]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   clean_text  89999 non-null  object 
 1   category    90000 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [8]:
dataset.isnull().sum()


clean_text    1
category      0
dtype: int64

In [9]:
is_NaN = dataset.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dataset[row_has_NaN]

print(rows_with_NaN)

    clean_text  category
148        NaN       0.0


In [10]:
dataset=dataset.dropna()

In [11]:
dataset = dataset.reset_index()

In [12]:
dataset.shape

(89999, 3)

In [13]:
dataset.head()

,index,clean_text,category
0,0,when modi promised “minimum government maximum...,-1.0
1,1,talk all the nonsense and continue all the dra...,0.0
2,2,what did just say vote for modi welcome bjp t...,1.0
3,3,asking his supporters prefix chowkidar their n...,1.0
4,4,answer who among these the most powerful world...,1.0


In [14]:
dataset =dataset.drop(['index'], axis = 1)

In [15]:
#cleaning the text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 89999):
  tweet = re.sub("[^a-zA-Z]", " ", dataset['clean_text'][i])
  tweet = tweet.lower()
  tweet = tweet.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  tweet = [ps.stem(word) for word in tweet if not word in set(all_stopwords)]
  tweet = ' '.join(tweet)
  corpus.append(tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
#creating bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

In [17]:
X.shape

(89999, 3000)

In [18]:
#slpiiting data into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [19]:
#naive bayes
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [20]:
#predicting test set result
y_pred = classifier.predict(X_test)

In [21]:
#making confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2385  953  567]
 [ 481 5299  292]
 [2079 2065 3879]]


0.6423888888888889

In [22]:
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)
print(np.mean(recall))
print(np.mean(precision))

0.6556449190242524
0.6460446682355377


In [23]:
f1Score = 2 * precision * recall / (precision + recall)
print(np.mean(f1Score))

0.627821329863378
